## Bibliotecas

In [16]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Baixando a base de dados do GitHub

In [2]:
!git clone https://github.com/vandelson-emf/Dataset.git

Cloning into 'Dataset'...
remote: Enumerating objects: 2164, done.
remote: Counting objects: 100% (2164/2164), done.
remote: Compressing objects: 100% (2124/2124), done.
remote: Total 2164 (delta 40), reused 2164 (delta 40), pack-reused 0 (from 0)
Receiving objects: 100% (2164/2164), 25.97 MiB | 29.51 MiB/s, done.
Resolving deltas: 100% (40/40), done.


In [3]:
pistachio_xls = "/content/Dataset/Pistachio_Image_Dataset/Pistachio_16_Features_Dataset/Pistachio_16_Features_Dataset.xls"

## Tratamento dos dados

In [6]:
df = pd.read_excel(pistachio_xls)
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4,Class
0,63391,1568.405,390.3396,236.7461,0.7951,284.0984,0.8665,73160,0.6394,1.6488,0.3238,0.7278,0.0062,0.0037,0.5297,0.8734,Kirmizi_Pistachio
1,68358,1942.187,410.8594,234.7525,0.8207,295.0188,0.8765,77991,0.6772,1.7502,0.2277,0.7181,0.0060,0.0034,0.5156,0.9024,Kirmizi_Pistachio
2,73589,1246.538,452.3630,220.5547,0.8731,306.0987,0.9172,80234,0.7127,2.0510,0.5951,0.6767,0.0061,0.0030,0.4579,0.9391,Kirmizi_Pistachio
3,71106,1445.261,429.5291,216.0765,0.8643,300.8903,0.9589,74153,0.7028,1.9879,0.4278,0.7005,0.0060,0.0030,0.4907,0.9755,Kirmizi_Pistachio
4,80087,1251.524,469.3783,220.9344,0.8823,319.3273,0.9657,82929,0.7459,2.1245,0.6425,0.6803,0.0059,0.0028,0.4628,0.9833,Kirmizi_Pistachio


In [7]:
df.shape

(2148, 17)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           2148 non-null   int64  
 1   PERIMETER      2148 non-null   float64
 2   MAJOR_AXIS     2148 non-null   float64
 3   MINOR_AXIS     2148 non-null   float64
 4   ECCENTRICITY   2148 non-null   float64
 5   EQDIASQ        2148 non-null   float64
 6   SOLIDITY       2148 non-null   float64
 7   CONVEX_AREA    2148 non-null   int64  
 8   EXTENT         2148 non-null   float64
 9   ASPECT_RATIO   2148 non-null   float64
 10  ROUNDNESS      2148 non-null   float64
 11  COMPACTNESS    2148 non-null   float64
 12  SHAPEFACTOR_1  2148 non-null   float64
 13  SHAPEFACTOR_2  2148 non-null   float64
 14  SHAPEFACTOR_3  2148 non-null   float64
 15  SHAPEFACTOR_4  2148 non-null   float64
 16  Class          2148 non-null   object 
dtypes: float64(14), int64(2), object(1)
memory usage: 28

In [8]:
df.describe()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4
count,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000,2148.000000
mean,79950.655493,1425.971751,446.248968,238.311842,0.840219,317.919173,0.940093,85015.839851,0.716067,1.898154,0.569172,0.713077,0.005701,0.003017,0.510461,0.955239
std,13121.737799,375.565503,32.445304,30.310695,0.048759,26.908600,0.050452,13154.919327,0.052532,0.240100,0.212760,0.044547,0.000817,0.000340,0.063942,0.051840
min,29808.000000,858.363000,320.344500,133.509600,0.504900,194.814600,0.588000,37935.000000,0.427200,1.158500,0.062800,0.476000,0.004000,0.002400,0.226600,0.620400
25%,71936.750000,1170.996250,426.508750,217.875825,0.817500,302.642850,0.919850,76467.000000,0.687000,1.736375,0.371250,0.681550,0.005200,0.002800,0.464475,0.944000
50%,79905.500000,1262.785500,448.574750,236.416350,0.849650,318.965300,0.954150,85075.500000,0.726500,1.896250,0.643400,0.710700,0.005600,0.003000,0.505150,0.973100
75%,89030.500000,1607.906250,468.509400,257.760150,0.875200,336.685525,0.976925,93893.500000,0.753600,2.067025,0.744125,0.741650,0.006100,0.003200,0.550075,0.987300
max,124008.000000,2755.049100,541.966100,383.046100,0.946000,397.356100,0.995100,132478.000000,0.820400,3.085800,0.933600,0.877900,0.013100,0.005300,0.770600,0.999000


In [10]:
# Verificando se há dados nulos
df.isnull().sum()

,0
AREA,0
PERIMETER,0
MAJOR_AXIS,0
MINOR_AXIS,0
ECCENTRICITY,0
EQDIASQ,0
SOLIDITY,0
CONVEX_AREA,0
EXTENT,0
ASPECT_RATIO,0


In [11]:
# Verificando se há dados duplicados
df.duplicated().sum()

0

In [12]:
# Verificando a quantidade de dados únicos em cada coluna
df.nunique()

,0
AREA,2101
PERIMETER,2144
MAJOR_AXIS,2143
MINOR_AXIS,2147
ECCENTRICITY,1172
EQDIASQ,2101
SOLIDITY,1070
CONVEX_AREA,2102
EXTENT,1222
ASPECT_RATIO,1922


In [13]:
# Criando df (X), excluindo a coluna Class que será o target (y)
x = df.drop('Class', axis=1)

x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   AREA           2148 non-null   int64  
 1   PERIMETER      2148 non-null   float64
 2   MAJOR_AXIS     2148 non-null   float64
 3   MINOR_AXIS     2148 non-null   float64
 4   ECCENTRICITY   2148 non-null   float64
 5   EQDIASQ        2148 non-null   float64
 6   SOLIDITY       2148 non-null   float64
 7   CONVEX_AREA    2148 non-null   int64  
 8   EXTENT         2148 non-null   float64
 9   ASPECT_RATIO   2148 non-null   float64
 10  ROUNDNESS      2148 non-null   float64
 11  COMPACTNESS    2148 non-null   float64
 12  SHAPEFACTOR_1  2148 non-null   float64
 13  SHAPEFACTOR_2  2148 non-null   float64
 14  SHAPEFACTOR_3  2148 non-null   float64
 15  SHAPEFACTOR_4  2148 non-null   float64
dtypes: float64(14), int64(2)
memory usage: 268.6 KB


In [17]:
# Normalizando os dados

ss = StandardScaler()
x_normalizado = pd.DataFrame(ss.fit_transform(x),columns=x.columns)

x_normalizado.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,ROUNDNESS,COMPACTNESS,SHAPEFACTOR_1,SHAPEFACTOR_2,SHAPEFACTOR_3,SHAPEFACTOR_4
0,-1.262296,0.379338,-1.723590,-0.051668,-0.925560,-1.257169,-1.459003,-0.901457,-1.459763,-1.038784,-1.153550,0.330588,0.610410,2.010131,0.300945,-1.579031
1,-0.883675,1.374821,-1.090999,-0.117456,-0.400406,-0.851241,-1.260749,-0.534133,-0.740038,-0.616362,-1.605338,0.112790,0.365630,1.126973,0.080381,-1.019493
2,-0.484931,-0.477881,0.188485,-0.585974,0.674518,-0.439385,-0.453857,-0.363587,-0.064106,0.636739,0.121894,-0.816778,0.488020,-0.050572,-0.822209,-0.311387
3,-0.674203,0.051373,-0.515445,-0.733752,0.493996,-0.632989,0.372861,-0.825955,-0.252605,0.373871,-0.664622,-0.282389,0.365630,-0.050572,-0.309125,0.390931
4,0.010393,-0.464602,0.713037,-0.573444,0.863245,0.052342,0.507673,-0.158673,0.568033,0.942933,0.344732,-0.735946,0.243241,-0.639344,-0.745559,0.541427


In [18]:
# Criando df (Y) apenas com os dados da coluna Class

y = pd.DataFrame(df['Class'])

y.head()

,Class
0,Kirmizi_Pistachio
1,Kirmizi_Pistachio
2,Kirmizi_Pistachio
3,Kirmizi_Pistachio
4,Kirmizi_Pistachio


In [21]:
# Transformando os dados categóricos de Y
le = LabelEncoder()

y = le.fit_transform(y)

y

array([0, 0, 0, ..., 1, 1, 1])

In [22]:
# Convertendo para Tensor
X = torch.tensor(x_normalizado.values, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

# Separando dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Criando TensorDataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Criando DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

## Criando a rede neural

In [26]:
class PistachioClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(PistachioClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size*2)
        self.relu = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size*2, hidden_size*4)
        self.relu = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size*4, hidden_size*2)
        self.relu = nn.ReLU()
        self.fc5 = nn.Linear(hidden_size*2, hidden_size)
        self.relu = nn.ReLU()
        self.fc6 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc6(out)
        return out



## Aplicando o modelo sobre os dados

In [27]:
# Parâmetros
input_size = X_train.shape[1]  # Número de features
hidden_size = 64
num_classes = 2 # Número de Classes (Kirmizi_Pistachio, Siit_Pistachio)

model = PistachioClassifier(input_size, hidden_size, num_classes)

# Definindo função de perda e otimizador
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Loop de Treino do Modelo
num_epochs = 200
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


# Loop de Avaliação do Modelo
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Acurácia do Modelo para a base de teste: {accuracy:.2f} %')

Epoch [1/200], Loss: 0.4544
Epoch [2/200], Loss: 0.4117
Epoch [3/200], Loss: 0.2095
Epoch [4/200], Loss: 0.1542
Epoch [5/200], Loss: 0.5056
Epoch [6/200], Loss: 0.1939
Epoch [7/200], Loss: 0.4475
Epoch [8/200], Loss: 0.2781
Epoch [9/200], Loss: 0.2836
Epoch [10/200], Loss: 0.4977
Epoch [11/200], Loss: 0.1559
Epoch [12/200], Loss: 0.4090
Epoch [13/200], Loss: 0.4007
Epoch [14/200], Loss: 0.6060
Epoch [15/200], Loss: 0.2204
Epoch [16/200], Loss: 0.3843
Epoch [17/200], Loss: 0.2002
Epoch [18/200], Loss: 0.1652
Epoch [19/200], Loss: 0.3612
Epoch [20/200], Loss: 0.3088
Epoch [21/200], Loss: 0.2251
Epoch [22/200], Loss: 0.2845
Epoch [23/200], Loss: 0.2894
Epoch [24/200], Loss: 0.2812
Epoch [25/200], Loss: 0.1718
Epoch [26/200], Loss: 0.3006
Epoch [27/200], Loss: 0.4429
Epoch [28/200], Loss: 0.2529
Epoch [29/200], Loss: 0.2261
Epoch [30/200], Loss: 0.1947
Epoch [31/200], Loss: 0.1794
Epoch [32/200], Loss: 0.2448
Epoch [33/200], Loss: 0.1405
Epoch [34/200], Loss: 0.2224
Epoch [35/200], Loss: 0